# Motivation 

## 2Design of Expressions in Blaze

The class hierarchy of the `Add` expression in Blaze is shown below:

![](https://dl.dropboxusercontent.com/u/97258109/Screens/S2850.png)

## The Node

The `Node` class represents a node in a tree. It is a base class for `Expr` and holds all tree traversal functions that are independent of tabular or array computation. Note that all of the special operations like `__add__` are not actually defined in the `Node` object except for as follows:

```python
def __rmul__(self, other):
    return self._rmul(other)
    
def __add__(self, other):
    return self._add(other)
```

So it is always the other function that is determining the action of the node tree. 

Method | Description
-- | --
`_leaves` | All nodes without inputs. Leaves are returned in order
`_inputs, _args` | `_args` retrives the `slot` attributes and `_inputs` retrives the `__inputs__`. They will often be `_child`. `__inputs__` will also be a subset of `__slots__` because they are attributes as well. 
`path(a,b)` | Function that shows a path of nodes from a to b 

In [4]:
from blaze.expr.core import Node

```python
class Node(object):
...
    __init__(self, *args, **kwargs):
        assert frozenset(kwargs).issubset(self.__slots__)
        
        for slot, arg in zip(self.__slots__[1:], args):
        #ignore the `self` argument. set the attribute to arg
            setattr(self, slot, arg)

        for key, value in kwargs.items():
            setattr(self, key, value) ```

`__slot__()` has not yet been defined in this object so we cannot call this object. It does, however, do some handling of the `slots` object.

It also defines the `__hash__`, `_traverse`, and `_subs` function. 

In [9]:
from blaze.expr import symbol
t = symbol('t', 'var * {name: string, amount: int, id: int}')
expr = t.amount * 3
expr._subs({3: 4, 'amount': 'id'})

t.id * 4

In [11]:
type(expr)

blaze.expr.arithmetic.Mult

In [14]:
from blaze import Symbol, Field
t = Symbol('t', 'var * {id: int, name: string, amount: int}')
amounts = Field(t, 'amount')
amounts, amounts._inputs, amounts._args

(t.amount, (t,), (t, 'amount'))

In [19]:
import pandas as pd

(23, ) + (22,)

(23, 22)

## Example of `path`

Path provides a path of nodes from `a` to `b`. 

  ```python
    >>> from blaze.expr import symbol
    >>> t = symbol('t', 'var * {name: string, amount: int, id: int}')
    >>> expr = t.amount.sum()
    >>> list(path(expr, t))
    [sum(t.amount), t.amount, t]```

# Expressions.py

This is the base class of all `Blaze` expressions such as `Join`, `Sort`, etc. There are also a number of important classes/functios here. 

| Class | Description 
-- | --
Symbol | This is the leaf of a blaze expression. It inherets the `Expr` object. It's slots are `_name`, `dshape`, and `_token`. 
ElemWise | Parent of `Expr`. Used for Element wise operations. Only sets the dshape 
Field | This is a single field from an expression. Stored in the `_name` attribute. Based from elemwise. Allows us to get a single field via `points.x.dshape

In [25]:
from blaze import symbol
points = symbol('points', '5 * 3 * {x: int32, y: int32}')
type(points.x), points.x.dshape, type(points.x.dshape), points._token

(blaze.expr.expressions.Field,
 dshape("5 * 3 * int32"),
 datashape.coretypes.DataShape,
 None)

## Expr object

Base class of all expressions. 

# Misc 

The lower case operations work as follows

```python
@memoize(cache=_symbol_cache, key=_symbol_key)
def symbol(name, dshape, token=None):
    return Symbol(name, dshape, token=token)
    
 ```

# Interactive object 

`interactive.py` has `Data` function and `InteractiveSymbol` class. 

In [1]:
from blaze import Data

In [4]:
DATA_DIR = '/Users/Will/Data/blaze_data/'
df = Data('sqlite:////' + DATA_DIR + 'lahman2013.sqlite').Batting

In [11]:
df[df.yearID > 2012]

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
0,aardsda01,2013,1,NYN,NL,43,43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,abadfe01,2013,1,WAS,NL,39,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
2,abreuto01,2013,1,SFN,NL,53,53,138,21,37,12,3,2,14,0,2,6,33,1,1,1,1,5,NaN
3,ackledu01,2013,1,SEA,AL,113,113,384,40,97,18,2,4,31,2,3,37,72,1,1,4,1,6,NaN
4,adamsda02,2013,1,NYA,AL,43,43,140,10,27,5,1,2,13,0,0,9,43,0,2,1,0,4,NaN
5,adamsma01,2013,1,SLN,NL,108,108,296,46,84,14,0,17,51,0,1,23,80,0,0,0,0,9,NaN
6,adamsmi03,2013,1,PHI,NL,28,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
7,adducji02,2013,1,TEX,AL,17,17,31,2,8,1,0,0,0,2,0,3,9,0,0,0,0,0,NaN
8,adriaeh01,2013,1,SFN,NL,9,9,18,3,4,1,0,1,3,0,0,1,5,0,0,1,0,1,NaN
9,affelje01,2013,1,SFN,NL,39,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [12]:
type(df[df.yearID > 2012])

blaze.expr.expressions.Selection

In [13]:
type(df[:5])

blaze.expr.expressions.Slice

In [14]:
df[:5]

Data:       Engine(sqlite://///Users/Will/Data/blaze_data/lahman2013.sqlite)
Expr:       _3.Batting[:5]
DataShape:  5 * {
  playerID: ?string,
  yearID: ?int32,
  stint: ?int32,
  teamID: ?string,
  lgID: ?string,
  G: ?int32,
  ...

## Data Object

`blaze.interactive.Data(data, dshape=None, name=None, fields=None, columns=None, schema=None, **kwargs)`

Presents a familar view onto a variety of forms of data. Provides interactive expeirnece to using Blaze's abstract expressions.

`data` is an object with a `discover` and `compute` implementation. 

# Questions

1. What does `_token` do? It really seems to do nothing